## This notebook runs a simulation of a rope using ANCF elements to model the flexibility and the DCA to form and solve the equations of motion

In [1]:
import math
import pickle 
import numpy as np
import scipy as sp
import sympy as sym
import matplotlib.pyplot as plt
import MBstructs as MB
import MultiBodyFuncts as MBF
import dca_recursive
import contextlib

from numpy.linalg import inv
from scipy.integrate import odeint

%matplotlib inline

In [2]:
@contextlib.contextmanager
def printoptions(*args, **kwargs):
    original = np.get_printoptions()
    np.set_printoptions(*args, **kwargs)
    yield 
    np.set_printoptions(**original)

In [3]:
# system specific kinematic sweep 
def kinematics(bodies,q,u):
    
    for body,theta,omega in zip(bodies,np.cumsum(q),np.cumsum(u)):
        body.omega = omega
        body.CBN = MBF.DCM(theta)

In [10]:
q0GEBF = [[1, 2, 3] for i in range(nGEBF)]
u0GEBF = [[11, 22, 33] for i in range(nGEBF)]

q0KIN =  [[np.pi/2]] + [[0.0 for i in range(ndofsKIN)] for i in range(nJKIN-1)]
u0KIN =  [[0] for i in range(nJKIN)]

q0 = np.array([q for pos in q0KIN+q0GEBF for q in pos]).squeeze()
u0 = np.array([q for pos in u0KIN+u0GEBF for q in pos]).squeeze()
state0 = np.hstack((q0,u0))

print(state0)

[  1.57079633   1.           2.           3.           1.           2.           3.
   1.           2.           3.           1.           2.           3.           1.
   2.           3.           1.           2.           3.           1.           2.
   3.           1.           2.           3.           1.           2.           3.
   1.           2.           3.           1.           2.           3.           1.
   2.           3.           0.          11.          22.          33.          11.
  22.          33.          11.          22.          33.          11.          22.
  33.          11.          22.          33.          11.          22.          33.
  11.          22.          33.          11.          22.          33.          11.
  22.          33.          11.          22.          33.          11.          22.
  33.          11.          22.          33.        ]


In [ ]:
qstate0.pop()

In [4]:
def simulate(state, t, nbodies, bodies, joints, BC1, BC2):
    """
    
    """ 
    q = state[:nbodies]
    u = state[nbodies:]
    
    # update state depedent quantities (all quantities for GEBF so just re-initialize)
    bodiesGEBF = [MB.GEBF_Element(E, I, r, rho, l, q)  for q in q0GEBF]     

    # DCA returns the eddot vector
    sol = dca_recursive(nbodies,0,bodies,joints,BC1,BC2)
    
    #loop to fill d_dt with the acceleration values
    d_dt=np.zeros((2*n))
    for j in range(0,n):
        if j == 0:
            A1=ll.pop(0)
            d_dt[j+n]=np.dot(np.transpose(joints[j].P),A1)
        else:
            A2= ll.pop(0)
            A1=ll.pop(0)
            d_dt[j+n]=np.dot(np.transpose(joints[j].P),(A1-A2))
    
    #add the velocities to d_dt and return to the integrator
    d_dt[:n]=state[n:]
    return d_dt 

### System Variables

In [5]:
# number of bodies and GEBF elements and kinematic joints
nGEBF = 4
nRIGID = 0
nbodies = nGEBF + nRIGID

# number of degrees of freedom for the 
# 2D-GEBF element with a single planar rotation and two planar displacements (2 nodes)
ndofsGEBF = 3
ndofsKIN  = 1 # (one pin joint)

### Initial Conditions

In [6]:
# start the rope from the horizontal 
# Compute initial generalized coordinates
# q = [theta1, u_1x, u_1y, theta2, u_2x','u_2y']
# u = q_dot
# state = [q, u]
q0GEBF = [[0.0 for i in range(ndofsGEBF)] for i in range(nGEBF)]
u0GEBF = [[0.0 for i in range(ndofsGEBF)] for i in range(nGEBF)]

q0KIN =  [[np.pi/2]] + [[0.0 for i in range(ndofsKIN)] for i in range(nJKIN-1)]
u0KIN =  [[0.0 for i in range(ndofsKIN)] for i in range(nJKIN)]

q0 = np.array([q for pos in q0KIN+q0GEBF for q in pos]).squeeze()
u0 = np.array([q for pos in u0KIN+u0GEBF for q in pos]).squeeze()
state0 = np.hstack((q0,u0))
# print(state0)

### Initilize the joints and bodies of the system 
12 GEBF bodies hooked to the ground by one kinematic joint

In [7]:
# Create a list of Joints
jointsKIN = [MB.Joint('revolute2D') for i in range(nJKIN)]
jointsGEBF = [MB.Joint('gebf') for i in range (nGEBF-1)]
joints = jointsKIN + jointsGEBF

# GEBF-Properties
# Physical Properties
A   =  0.0018
I   =  1.215e-8
L   =  1.2
r   =  math.sqrt(A/math.pi)
l   =  L/nGEBF

# Material Properties
E   =  0.7e6
rho =  5540

bodiesGEBF = [MB.Body() for i in range(nGEBF)]
for body in bodiesGEBF:
    body.initialize('gebf', E, A, I, r, rho, l)

In [9]:
# # For systems with bodies of different type:
# #   bodies are numbered from left to right

# # initialize(bodies, y, n) (rigid bodies)

# # Material Properties
# E   =  0.7e6
# rho =  5540

# # update state depedent quantities (all quantities for GEBF so just re-initialize)
# bodiesGEBF = [MB.GEBF_Element2D(E, A, I, r, rho, l, q)  for q in q0GEBF]
# bodies = bodiesGEBF

# # # DCA returns the eddot vector
# # sol = dca_recursive.solve(nbodies,0,bodies,joints,2,1)

# # uKIN = np.dot(np.transpose(joints[0].P*sol.pop(0)))
# # uGEBF = [sol.pop(0)[2:6].reshape(1,4) for i in range(5)]

### Integration

In [ ]:
# Length of time of the simulation
tspan = np.arange(0,5,.001)
# odeint is the numerical integrator used
state = odeint(simulate,state0,tspan,(n,0,bodies,joints,2,1))

In [ ]:
# Energy Calculation
#--------------------
#The energy of the system is calculated and plotted

energy=MBF.PendEnergy(y,elements)
KE=energy[:,0]
PE=energy[:,1]
TE=energy[:,2]

plt.plot(t,TE-TE[0])
plt.xlabel("Time [s]")
plt.ylabel("energy")
plt.title("System Energy")
plt.show()

plt.plot(t,PE,Time,KE)
plt.xlabel("Time[s]")
plt.ylabel("energy")
plt.title("Kinetic and Potential Energy")
plt.show

In [ ]:
# Solution Plot
#--------------------
plt.plot(Time,yy[:,:n])
plt.xlabel("Time [s]")
plt.ylabel("Generalized Coordinates [Rad]")
plt.title("System Response")

plt.show()

plt.plot(Time,yy[:,n:])
plt.xlabel(("Time[s]"))
plt.ylabel(("Generalized Speeds [Rad/s]"))
plt.title("System Response")

plt.show()

In [ ]:
25%2

In [ ]:
import math
print(math.trunc(40/2))
print(40//2)

In [ ]:
test = [np.linspace(1,4,4).reshape(4,1) for i in range(5)]

In [ ]:
test

In [ ]:
[test.pop(0)[2:5].reshape(1,2) for i in range(5)]

In [ ]:
l = 1
r = np.array([l/2,0,0])

In [ ]:
np.hstack(([1,0,0],np.vstack(([r[1], r[0]],np.eye(2)))))

In [ ]:
np.array([r[1], r[0]])

In [ ]:
np.hstack((np.array([[1],[0],[0]]),np.vstack(([r[1], r[0]],np.eye(2)))))

In [ ]:
I  =500
m = 23

In [ ]:
np.hstack(((np.array([[I],[0],[0]]),np.vstack((np.zeros((1,2)),m*np.eye(2))))))